图片预处理及增强

In [1]:
from keras.applications import vgg16
from keras.preprocessing.image import ImageDataGenerator

def img_preprocessing(val_path,tra_path):
    img_height, img_width=(224,224)
    vgg_train_datagen = ImageDataGenerator(zoom_range=[0.7,1.3],
                                           rotation_range=15,
                                           width_shift_range=0.2,
                                           height_shift_range=0.2,
                                           preprocessing_function=vgg16.preprocess_input)
    vgg_validation_datagen = ImageDataGenerator(preprocessing_function=vgg16.preprocess_input)

    print("")
    print("vgg train/validation dataset loading")
    print("------------------------------------")
    print("")
    vgg_train_generator = vgg_train_datagen.flow_from_directory(
            tra_path,
            target_size=( img_height, img_width),
            batch_size=32,
            class_mode='categorical')
    vgg_validation_generator = vgg_validation_datagen.flow_from_directory(
            val_path,
            target_size=( img_height, img_width),
            batch_size=32,
            class_mode='categorical',
            shuffle=False)
    return vgg_train_generator,vgg_validation_generator

C:\Users\Q\Anaconda2\envs\tensorflow-gpu\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


模型搭建

In [2]:
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.models import Model
from keras.layers import GlobalAveragePooling2D,Dense,Dropout
from keras import backend as K
from keras import optimizers
from keras import regularizers
from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping

def vgg_train(path,vgg_train_generator,vgg_validation_generator,lr,dec):
    vgg_base_model = VGG16(weights='imagenet', include_top=False)
    x_vgg = vgg_base_model.output
    x_vgg = GlobalAveragePooling2D()(x_vgg)
    x_vgg = Dropout(0.5)(x_vgg)
    
    predictions = Dense(10, activation='softmax')(x_vgg)
    vgg_model = Model(inputs=vgg_base_model.input, outputs=predictions)
    for layer in vgg_base_model.layers[:11]:
        layer.trainable = False
    for layer in vgg_base_model.layers[11:]:
        layer.trainable = True    
    sgd = optimizers.SGD(lr=lr, decay=dec, momentum=0.9, nesterov=True)
    vgg_model.compile(optimizer=sgd, loss='categorical_crossentropy',metrics=['categorical_accuracy'])
    CHECKPOINT_MODEL_SAVE_PATH = os.path.join(path,"vgg_best_model_lr-4_decay-6{epoch:04d}_{val_loss:.4f}.hdf5")
    model_check_point =ModelCheckpoint(CHECKPOINT_MODEL_SAVE_PATH,monitor='val_loss',verbose=1,save_best_only=True,save_weights_only=False,period=1) 
    model_early_stop=EarlyStopping(monitor='val_loss', patience=5, verbose=1, mode='min')
    print("")
    print("")
    print("vgg training")
    print("------------------------------------")
    print("")
    vgg_model.fit_generator(
                            vgg_train_generator,
                            epochs=30,
                            validation_data=vgg_validation_generator,
                            callbacks=[model_check_point,model_early_stop]
                            )

模型训练

In [3]:
train_folder = 'train'
test_folder = 'test'
my_train_folder='my_train'
my_validation_folder = 'my_validation'
path_num=9

import os
import glob

for i in range(path_num):
    path_i = os.path.join('t' + str(i))
    v_path=os.path.join(path_i,my_validation_folder)
    t_path=os.path.join(path_i,my_train_folder)
    print(path_i,v_path,t_path)
    print("------------------------------------")
    vgg_train_generator,vgg_validation_generator=img_preprocessing(v_path,t_path)
    vgg_train (path_i,vgg_train_generator,vgg_validation_generator,1e-4,1e-6)

t0 t0\my_validation t0\my_train
------------------------------------

vgg train/validation dataset loading
------------------------------------

Found 19725 images belonging to 10 classes.
Found 2699 images belonging to 10 classes.


vgg training
------------------------------------

Epoch 1/30
616/617 [============================>.] - ETA: 0s - loss: 2.3847 - categorical_accuracy: 0.1090
Epoch 00001: val_loss improved from inf to 2.27627, saving model to t0\vgg_best_model_lr-4_decay-60001_2.2763.hdf5
617/617 [==============================] - 236s 383ms/step - loss: 2.3845 - categorical_accuracy: 0.1090 - val_loss: 2.2763 - val_categorical_accuracy: 0.1486
Epoch 2/30
616/617 [============================>.] - ETA: 0s - loss: 2.1098 - categorical_accuracy: 0.2234
Epoch 00002: val_loss improved from 2.27627 to 1.56427, saving model to t0\vgg_best_model_lr-4_decay-60002_1.5643.hdf5
617/617 [==============================] - 226s 366ms/step - loss: 2.1094 - categorical_accuracy: 0.2237 -

Epoch 8/30
645/646 [============================>.] - ETA: 0s - loss: 0.3327 - categorical_accuracy: 0.8972
Epoch 00008: val_loss did not improve
646/646 [==============================] - 235s 364ms/step - loss: 0.3326 - categorical_accuracy: 0.8972 - val_loss: 0.9604 - val_categorical_accuracy: 0.7188
Epoch 9/30
645/646 [============================>.] - ETA: 0s - loss: 0.2509 - categorical_accuracy: 0.9255
Epoch 00009: val_loss did not improve
646/646 [==============================] - 233s 361ms/step - loss: 0.2508 - categorical_accuracy: 0.9255 - val_loss: 1.0258 - val_categorical_accuracy: 0.7442
Epoch 10/30
645/646 [============================>.] - ETA: 0s - loss: 0.1971 - categorical_accuracy: 0.9420
Epoch 00010: val_loss did not improve
646/646 [==============================] - 233s 361ms/step - loss: 0.1970 - categorical_accuracy: 0.9420 - val_loss: 1.1173 - val_categorical_accuracy: 0.7583
Epoch 11/30
645/646 [============================>.] - ETA: 0s - loss: 0.1554 - cate

Epoch 8/30
634/635 [============================>.] - ETA: 0s - loss: 0.2613 - categorical_accuracy: 0.9211
Epoch 00008: val_loss improved from 0.74155 to 0.64985, saving model to t3\vgg_best_model_lr-4_decay-60008_0.6499.hdf5
635/635 [==============================] - 233s 367ms/step - loss: 0.2612 - categorical_accuracy: 0.9211 - val_loss: 0.6499 - val_categorical_accuracy: 0.7761
Epoch 9/30
634/635 [============================>.] - ETA: 0s - loss: 0.2064 - categorical_accuracy: 0.9388
Epoch 00009: val_loss did not improve
635/635 [==============================] - 234s 368ms/step - loss: 0.2063 - categorical_accuracy: 0.9389 - val_loss: 0.9229 - val_categorical_accuracy: 0.7029
Epoch 10/30
634/635 [============================>.] - ETA: 0s - loss: 0.1685 - categorical_accuracy: 0.9501
Epoch 00010: val_loss did not improve
635/635 [==============================] - 233s 367ms/step - loss: 0.1684 - categorical_accuracy: 0.9501 - val_loss: 1.0412 - val_categorical_accuracy: 0.6687
Epo

Epoch 4/30
627/628 [============================>.] - ETA: 0s - loss: 0.4547 - categorical_accuracy: 0.8578
Epoch 00004: val_loss improved from 0.67509 to 0.41274, saving model to t5\vgg_best_model_lr-4_decay-60004_0.4127.hdf5
628/628 [==============================] - 233s 370ms/step - loss: 0.4550 - categorical_accuracy: 0.8578 - val_loss: 0.4127 - val_categorical_accuracy: 0.8760
Epoch 5/30
627/628 [============================>.] - ETA: 0s - loss: 0.2821 - categorical_accuracy: 0.9130
Epoch 00005: val_loss improved from 0.41274 to 0.37483, saving model to t5\vgg_best_model_lr-4_decay-60005_0.3748.hdf5
628/628 [==============================] - 234s 372ms/step - loss: 0.2821 - categorical_accuracy: 0.9129 - val_loss: 0.3748 - val_categorical_accuracy: 0.8786
Epoch 6/30
627/628 [============================>.] - ETA: 0s - loss: 0.2172 - categorical_accuracy: 0.9375
Epoch 00006: val_loss did not improve
628/628 [==============================] - 233s 371ms/step - loss: 0.2174 - catego

Epoch 5/30
608/609 [============================>.] - ETA: 0s - loss: 0.2959 - categorical_accuracy: 0.9063
Epoch 00005: val_loss improved from 0.45265 to 0.40082, saving model to t7\vgg_best_model_lr-4_decay-60005_0.4008.hdf5
609/609 [==============================] - 230s 378ms/step - loss: 0.2957 - categorical_accuracy: 0.9064 - val_loss: 0.4008 - val_categorical_accuracy: 0.8907
Epoch 6/30
608/609 [============================>.] - ETA: 0s - loss: 0.2342 - categorical_accuracy: 0.9292
Epoch 00006: val_loss did not improve
609/609 [==============================] - 232s 381ms/step - loss: 0.2340 - categorical_accuracy: 0.9292 - val_loss: 0.4352 - val_categorical_accuracy: 0.8787
Epoch 7/30
608/609 [============================>.] - ETA: 0s - loss: 0.1762 - categorical_accuracy: 0.9458
Epoch 00007: val_loss did not improve
609/609 [==============================] - 230s 378ms/step - loss: 0.1764 - categorical_accuracy: 0.9457 - val_loss: 0.4318 - val_categorical_accuracy: 0.8719
Epoc

Epoch 18/30
617/618 [============================>.] - ETA: 0s - loss: 0.0549 - categorical_accuracy: 0.9834
Epoch 00018: val_loss did not improve
618/618 [==============================] - 229s 370ms/step - loss: 0.0548 - categorical_accuracy: 0.9834 - val_loss: 0.3496 - val_categorical_accuracy: 0.9209
Epoch 19/30
617/618 [============================>.] - ETA: 0s - loss: 0.0546 - categorical_accuracy: 0.9836
Epoch 00019: val_loss improved from 0.31856 to 0.29081, saving model to t8\vgg_best_model_lr-4_decay-60019_0.2908.hdf5
618/618 [==============================] - 229s 371ms/step - loss: 0.0545 - categorical_accuracy: 0.9836 - val_loss: 0.2908 - val_categorical_accuracy: 0.9247
Epoch 20/30
617/618 [============================>.] - ETA: 0s - loss: 0.0504 - categorical_accuracy: 0.9847
Epoch 00020: val_loss did not improve
618/618 [==============================] - 229s 371ms/step - loss: 0.0504 - categorical_accuracy: 0.9847 - val_loss: 0.3628 - val_categorical_accuracy: 0.9224
E

In [3]:

train_folder = 'train'
test_folder = 'test'
my_train_folder='my_train'
my_validation_folder = 'my_validation'
path_num=9

import os
import glob

t=4
for i in range(path_num):
    path_i = os.path.join('t' + str(t))
    v_path=os.path.join(path_i,my_validation_folder)
    t_path=os.path.join(path_i,my_train_folder)
    print(path_i,v_path,t_path)
    print("------------------------------------")
    vgg_train_generator,vgg_validation_generator=img_preprocessing(v_path,t_path)
    vgg_train (path_i,vgg_train_generator,vgg_validation_generator,1e-4,1e-6)

t4 t4\my_validation t4\my_train
------------------------------------

vgg train/validation dataset loading
------------------------------------

Found 19330 images belonging to 10 classes.
Found 3094 images belonging to 10 classes.


vgg training
------------------------------------

Epoch 1/30
604/605 [============================>.] - ETA: 0s - loss: 2.3645 - categorical_accuracy: 0.1111
Epoch 00001: val_loss improved from inf to 2.28594, saving model to t4\vgg_best_model_lr-4_decay-60001_2.2859.hdf5
605/605 [==============================] - 234s 387ms/step - loss: 2.3644 - categorical_accuracy: 0.1111 - val_loss: 2.2859 - val_categorical_accuracy: 0.1312
Epoch 2/30
604/605 [============================>.] - ETA: 0s - loss: 2.2389 - categorical_accuracy: 0.1431
Epoch 00002: val_loss improved from 2.28594 to 2.07656, saving model to t4\vgg_best_model_lr-4_decay-60002_2.0766.hdf5
605/605 [==============================] - 223s 368ms/step - loss: 2.2387 - categorical_accuracy: 0.1432 -

604/605 [============================>.] - ETA: 0s - loss: 0.3607 - categorical_accuracy: 0.8865
Epoch 00007: val_loss improved from 0.59217 to 0.56250, saving model to t4\vgg_best_model_lr-4_decay-60007_0.5625.hdf5
605/605 [==============================] - 218s 361ms/step - loss: 0.3604 - categorical_accuracy: 0.8866 - val_loss: 0.5625 - val_categorical_accuracy: 0.8575
Epoch 8/30
604/605 [============================>.] - ETA: 0s - loss: 0.2682 - categorical_accuracy: 0.9173
Epoch 00008: val_loss improved from 0.56250 to 0.43888, saving model to t4\vgg_best_model_lr-4_decay-60008_0.4389.hdf5
605/605 [==============================] - 218s 360ms/step - loss: 0.2680 - categorical_accuracy: 0.9174 - val_loss: 0.4389 - val_categorical_accuracy: 0.8597
Epoch 9/30
604/605 [============================>.] - ETA: 0s - loss: 0.2203 - categorical_accuracy: 0.9320
Epoch 00009: val_loss improved from 0.43888 to 0.32899, saving model to t4\vgg_best_model_lr-4_decay-60009_0.3290.hdf5
605/605 [===

Epoch 2/30
604/605 [============================>.] - ETA: 0s - loss: 2.2358 - categorical_accuracy: 0.1467
Epoch 00002: val_loss improved from 2.24555 to 2.08954, saving model to t4\vgg_best_model_lr-4_decay-60002_2.0895.hdf5
605/605 [==============================] - 220s 364ms/step - loss: 2.2354 - categorical_accuracy: 0.1468 - val_loss: 2.0895 - val_categorical_accuracy: 0.2573
Epoch 3/30
604/605 [============================>.] - ETA: 0s - loss: 1.9212 - categorical_accuracy: 0.2899
Epoch 00003: val_loss improved from 2.08954 to 1.19241, saving model to t4\vgg_best_model_lr-4_decay-60003_1.1924.hdf5
605/605 [==============================] - 220s 363ms/step - loss: 1.9211 - categorical_accuracy: 0.2900 - val_loss: 1.1924 - val_categorical_accuracy: 0.6500
Epoch 4/30
604/605 [============================>.] - ETA: 0s - loss: 0.9591 - categorical_accuracy: 0.6760
Epoch 00004: val_loss improved from 1.19241 to 0.52135, saving model to t4\vgg_best_model_lr-4_decay-60004_0.5214.hdf5
6

Epoch 6/30
604/605 [============================>.] - ETA: 0s - loss: 0.2974 - categorical_accuracy: 0.9067
Epoch 00006: val_loss improved from 0.44860 to 0.38363, saving model to t4\vgg_best_model_lr-4_decay-60006_0.3836.hdf5
605/605 [==============================] - 221s 365ms/step - loss: 0.2973 - categorical_accuracy: 0.9068 - val_loss: 0.3836 - val_categorical_accuracy: 0.8920
Epoch 7/30
604/605 [============================>.] - ETA: 0s - loss: 0.2228 - categorical_accuracy: 0.9323
Epoch 00007: val_loss improved from 0.38363 to 0.36643, saving model to t4\vgg_best_model_lr-4_decay-60007_0.3664.hdf5
605/605 [==============================] - 220s 364ms/step - loss: 0.2226 - categorical_accuracy: 0.9324 - val_loss: 0.3664 - val_categorical_accuracy: 0.8988
Epoch 8/30
604/605 [============================>.] - ETA: 0s - loss: 0.1775 - categorical_accuracy: 0.9469
Epoch 00008: val_loss did not improve
605/605 [==============================] - 221s 365ms/step - loss: 0.1773 - catego

605/605 [==============================] - 227s 375ms/step - loss: 2.3791 - categorical_accuracy: 0.1102 - val_loss: 2.2879 - val_categorical_accuracy: 0.1484
Epoch 2/30
604/605 [============================>.] - ETA: 0s - loss: 2.2265 - categorical_accuracy: 0.1547
Epoch 00002: val_loss improved from 2.28793 to 1.96168, saving model to t4\vgg_best_model_lr-4_decay-60002_1.9617.hdf5
605/605 [==============================] - 223s 368ms/step - loss: 2.2264 - categorical_accuracy: 0.1550 - val_loss: 1.9617 - val_categorical_accuracy: 0.3229
Epoch 3/30
604/605 [============================>.] - ETA: 0s - loss: 1.5877 - categorical_accuracy: 0.4183
Epoch 00003: val_loss improved from 1.96168 to 0.78948, saving model to t4\vgg_best_model_lr-4_decay-60003_0.7895.hdf5
605/605 [==============================] - 223s 368ms/step - loss: 1.5867 - categorical_accuracy: 0.4186 - val_loss: 0.7895 - val_categorical_accuracy: 0.7576
Epoch 4/30
604/605 [============================>.] - ETA: 0s - loss:

Epoch 13/30
604/605 [============================>.] - ETA: 0s - loss: 0.1612 - categorical_accuracy: 0.9505
Epoch 00013: val_loss did not improve
605/605 [==============================] - 223s 368ms/step - loss: 0.1610 - categorical_accuracy: 0.9506 - val_loss: 0.5667 - val_categorical_accuracy: 0.8361
Epoch 00013: early stopping
t4 t4\my_validation t4\my_train
------------------------------------

vgg train/validation dataset loading
------------------------------------

Found 19330 images belonging to 10 classes.
Found 3094 images belonging to 10 classes.


vgg training
------------------------------------

Epoch 1/30
604/605 [============================>.] - ETA: 0s - loss: 2.4075 - categorical_accuracy: 0.1079
Epoch 00001: val_loss improved from inf to 2.29723, saving model to t4\vgg_best_model_lr-4_decay-60001_2.2972.hdf5
605/605 [==============================] - 227s 376ms/step - loss: 2.4073 - categorical_accuracy: 0.1079 - val_loss: 2.2972 - val_categorical_accuracy: 0.1328

In [4]:
t=5
for i in range(path_num):
    path_i = os.path.join('t' + str(t))
    v_path=os.path.join(path_i,my_validation_folder)
    t_path=os.path.join(path_i,my_train_folder)
    print(path_i,v_path,t_path)
    print("------------------------------------")
    vgg_train_generator,vgg_validation_generator=img_preprocessing(v_path,t_path)
    vgg_train (path_i,vgg_train_generator,vgg_validation_generator,1e-4,1e-6)

t5 t5\my_validation t5\my_train
------------------------------------

vgg train/validation dataset loading
------------------------------------

Found 20077 images belonging to 10 classes.
Found 2347 images belonging to 10 classes.


vgg training
------------------------------------

Epoch 1/30
627/628 [============================>.] - ETA: 0s - loss: 2.3270 - categorical_accuracy: 0.1489
Epoch 00001: val_loss improved from inf to 2.04882, saving model to t5\vgg_best_model_lr-4_decay-60001_2.0488.hdf5
628/628 [==============================] - 240s 381ms/step - loss: 2.3265 - categorical_accuracy: 0.1491 - val_loss: 2.0488 - val_categorical_accuracy: 0.3046
Epoch 2/30
627/628 [============================>.] - ETA: 0s - loss: 1.6633 - categorical_accuracy: 0.3866
Epoch 00002: val_loss improved from 2.04882 to 1.03724, saving model to t5\vgg_best_model_lr-4_decay-60002_1.0372.hdf5
628/628 [==============================] - 227s 362ms/step - loss: 1.6624 - categorical_accuracy: 0.3870 -

Epoch 10/30
627/628 [============================>.] - ETA: 0s - loss: 0.1322 - categorical_accuracy: 0.9619
Epoch 00010: val_loss did not improve
628/628 [==============================] - 228s 363ms/step - loss: 0.1321 - categorical_accuracy: 0.9619 - val_loss: 0.4313 - val_categorical_accuracy: 0.8909
Epoch 11/30
627/628 [============================>.] - ETA: 0s - loss: 0.1167 - categorical_accuracy: 0.9663
Epoch 00011: val_loss did not improve
628/628 [==============================] - 228s 363ms/step - loss: 0.1167 - categorical_accuracy: 0.9663 - val_loss: 0.4237 - val_categorical_accuracy: 0.8798
Epoch 12/30
627/628 [============================>.] - ETA: 0s - loss: 0.1112 - categorical_accuracy: 0.9700
Epoch 00012: val_loss improved from 0.38164 to 0.31523, saving model to t5\vgg_best_model_lr-4_decay-60012_0.3152.hdf5
628/628 [==============================] - 228s 363ms/step - loss: 0.1111 - categorical_accuracy: 0.9700 - val_loss: 0.3152 - val_categorical_accuracy: 0.8986
E

Epoch 2/30
627/628 [============================>.] - ETA: 0s - loss: 1.9746 - categorical_accuracy: 0.2607
Epoch 00002: val_loss improved from 2.16821 to 1.61285, saving model to t5\vgg_best_model_lr-4_decay-60002_1.6128.hdf5
628/628 [==============================] - 229s 365ms/step - loss: 1.9737 - categorical_accuracy: 0.2611 - val_loss: 1.6128 - val_categorical_accuracy: 0.4977
Epoch 3/30
627/628 [============================>.] - ETA: 0s - loss: 1.3567 - categorical_accuracy: 0.5141
Epoch 00003: val_loss improved from 1.61285 to 0.60365, saving model to t5\vgg_best_model_lr-4_decay-60003_0.6037.hdf5
628/628 [==============================] - 229s 364ms/step - loss: 1.3561 - categorical_accuracy: 0.5142 - val_loss: 0.6037 - val_categorical_accuracy: 0.8568
Epoch 4/30
627/628 [============================>.] - ETA: 0s - loss: 0.6709 - categorical_accuracy: 0.7739
Epoch 00004: val_loss improved from 0.60365 to 0.49097, saving model to t5\vgg_best_model_lr-4_decay-60004_0.4910.hdf5
6

Epoch 13/30
627/628 [============================>.] - ETA: 0s - loss: 0.1100 - categorical_accuracy: 0.9668
Epoch 00013: val_loss did not improve
628/628 [==============================] - 229s 365ms/step - loss: 0.1099 - categorical_accuracy: 0.9669 - val_loss: 0.4445 - val_categorical_accuracy: 0.8764
Epoch 14/30
627/628 [============================>.] - ETA: 0s - loss: 0.0942 - categorical_accuracy: 0.9707
Epoch 00014: val_loss did not improve
628/628 [==============================] - 230s 366ms/step - loss: 0.0941 - categorical_accuracy: 0.9708 - val_loss: 0.4846 - val_categorical_accuracy: 0.8722
Epoch 15/30
627/628 [============================>.] - ETA: 0s - loss: 0.0820 - categorical_accuracy: 0.9762
Epoch 00015: val_loss did not improve
628/628 [==============================] - 231s 368ms/step - loss: 0.0820 - categorical_accuracy: 0.9762 - val_loss: 0.4496 - val_categorical_accuracy: 0.8645
Epoch 00015: early stopping
t5 t5\my_validation t5\my_train
----------------------

628/628 [==============================] - 240s 382ms/step - loss: 2.3748 - categorical_accuracy: 0.1182 - val_loss: 2.2627 - val_categorical_accuracy: 0.1572
Epoch 2/30
627/628 [============================>.] - ETA: 0s - loss: 2.1668 - categorical_accuracy: 0.1836
Epoch 00002: val_loss improved from 2.26270 to 2.02526, saving model to t5\vgg_best_model_lr-4_decay-60002_2.0253.hdf5
628/628 [==============================] - 235s 374ms/step - loss: 2.1667 - categorical_accuracy: 0.1837 - val_loss: 2.0253 - val_categorical_accuracy: 0.2518
Epoch 3/30
627/628 [============================>.] - ETA: 0s - loss: 1.6553 - categorical_accuracy: 0.3993
Epoch 00003: val_loss improved from 2.02526 to 1.24241, saving model to t5\vgg_best_model_lr-4_decay-60003_1.2424.hdf5
628/628 [==============================] - 241s 384ms/step - loss: 1.6549 - categorical_accuracy: 0.3995 - val_loss: 1.2424 - val_categorical_accuracy: 0.6285
Epoch 4/30
627/628 [============================>.] - ETA: 0s - loss:

Epoch 10/30
627/628 [============================>.] - ETA: 0s - loss: 0.1479 - categorical_accuracy: 0.9558
Epoch 00010: val_loss did not improve
628/628 [==============================] - 238s 379ms/step - loss: 0.1479 - categorical_accuracy: 0.9558 - val_loss: 0.3850 - val_categorical_accuracy: 0.8828
Epoch 11/30
627/628 [============================>.] - ETA: 0s - loss: 0.1266 - categorical_accuracy: 0.9625
Epoch 00011: val_loss did not improve
628/628 [==============================] - 235s 374ms/step - loss: 0.1264 - categorical_accuracy: 0.9626 - val_loss: 0.4309 - val_categorical_accuracy: 0.8845
Epoch 12/30
627/628 [============================>.] - ETA: 0s - loss: 0.1082 - categorical_accuracy: 0.9669
Epoch 00012: val_loss did not improve
628/628 [==============================] - 234s 372ms/step - loss: 0.1081 - categorical_accuracy: 0.9670 - val_loss: 0.4707 - val_categorical_accuracy: 0.8722
Epoch 13/30
627/628 [============================>.] - ETA: 0s - loss: 0.0961 - ca